# Multiple Dispatch

In this notebook we'll explore **multiple dispatch**, which is a key feature of Julia.

Multiple dispatch makes software *generic* and *fast*!

#### Starting with the familiar

To understand multiple dispatch in Julia, let's start with what we've already seen.

We can declare functions in Julia without giving Julia any information about the types of the input arguments that function will receive:

In [1]:
square(x) = x^2

square (generic function with 1 method)

In [2]:
square(10)

100

In [3]:
square("Hello ")

"Hello Hello "

In [4]:
square([1,2,3])

LoadError: MethodError: no method matching ^(::Vector{Int64}, ::Int64)

[0mClosest candidates are:
[0m  ^([91m::Union{AbstractChar, AbstractString}[39m, ::Integer)
[0m[90m   @[39m [90mBase[39m [90mstrings\[39m[90m[4mbasic.jl:733[24m[39m
[0m  ^([91m::Rational[39m, ::Integer)
[0m[90m   @[39m [90mBase[39m [90m[4mrational.jl:491[24m[39m
[0m  ^([91m::LinearAlgebra.Hermitian[39m, ::Integer)
[0m[90m   @[39m [36mLinearAlgebra[39m [90mC:\Users\jwei\AppData\Local\Programs\Julia-1.9.0\share\julia\stdlib\v1.9\LinearAlgebra\src\[39m[90m[4msymmetric.jl:697[24m[39m
[0m  ...


#### Specifying the types of our input arguments

However, we also have the *option* to tell Julia explicitly what types our input arguments are allowed to have.

For example, let's write a function `f` that only takes `Number`s as inputs.

In [5]:
f(a::Integer, b::Integer) = "a and b are both integers"

f (generic function with 1 method)

In [6]:
f(3, 4)

"a and b are both integers"

In [7]:
f(1.2, 3.4)

LoadError: MethodError: no method matching f(::Float64, ::Float64)

But we can define that method!

In [8]:
f(a::Float64, b::Float64) = "a and b are both Float64s"

f (generic function with 2 methods)

In [9]:
f(1.2, 3.4)

"a and b are both Float64s"

### Basic dispatch

In [10]:
f(a, b) = "fallback"
f(a::Number, b::Number) = "a and b are both numbers"
f(a::Number, b) = "a is a number"
f(a, b::Number) = "b is a number"
f(a::Integer, b::Integer) = "a and b are both integers"

f (generic function with 6 methods)

In [11]:
methods(f)

# 6 methods for generic function "f" from Main:
 [1] f(a::Integer, b::Integer)
     @ In[10]:5
 [2] f(a::Float64, b::Float64)
     @ In[8]:1
 [3] f(a::Number, b::Number)
     @ In[10]:2
 [4] f(a::Number, b)
     @ In[10]:3
 [5] f(a, b::Number)
     @ In[10]:4
 [6] f(a, b)
     @ In[10]:1

In [12]:
f(1.5, 2)

"a and b are both numbers"

In [13]:
f(1, "bar")

"a is a number"

In [14]:
f(1, 2)

"a and b are both integers"

In [15]:
f("foo", [1,2])

"fallback"

In [16]:
f(1, 2, 3)

LoadError: MethodError: no method matching f(::Int64, ::Int64, ::Int64)

[0mClosest candidates are:
[0m  f(::Integer, ::Integer)
[0m[90m   @[39m [35mMain[39m [90m[4mIn[10]:5[24m[39m
[0m  f(::Number, ::Number)
[0m[90m   @[39m [35mMain[39m [90m[4mIn[10]:2[24m[39m
[0m  f(::Number, ::Any)
[0m[90m   @[39m [35mMain[39m [90m[4mIn[10]:3[24m[39m
[0m  ...


### Ambiguities

In [17]:
g(a::Int, b::Number) = 1
g(a::Number, b::Int) = 2

g (generic function with 2 methods)

In [18]:
g(1, 2.5)

1

In [19]:
g(1.5, 2)

2

In [20]:
g(1, 2)

LoadError: MethodError: g(::Int64, ::Int64) is ambiguous.

Candidates:
  g([90ma[39m::[1mInt64[22m, [90mb[39m::[1mNumber[22m)
[90m    @[39m [90mMain[39m [90m[4mIn[17]:1[24m[39m
  g([90ma[39m::[1mNumber[22m, [90mb[39m::[1mInt64[22m)
[90m    @[39m [90mMain[39m [90m[4mIn[17]:2[24m[39m

Possible fix, define
  g(::Int64, ::Int64)


In [21]:
g(x::Int, y::Int) = 3

g (generic function with 3 methods)

In [22]:
g(1, 2)

3

### "Diagonal" dispatch

In [23]:
f(a::T, b::T) where {T<:Number} = "a and b are both $(T)s"

f (generic function with 7 methods)

In [24]:
methods(f)

# 7 methods for generic function "f" from Main:
 [1] f(a::Integer, b::Integer)
     @ In[10]:5
 [2] f(a::Float64, b::Float64)
     @ In[8]:1
 [3] f(a::T, b::T) where T<:Number
     @ In[23]:1
 [4] f(a::Number, b::Number)
     @ In[10]:2
 [5] f(a::Number, b)
     @ In[10]:3
 [6] f(a, b::Number)
     @ In[10]:4
 [7] f(a, b)
     @ In[10]:1

In [25]:
f(big(1.5), big(2.5))

"a and b are both BigFloats"

In [26]:
f(big(1), big(2)) # <== integer rule is more specific

"a and b are both integers"

In [27]:
f(a::T, b::T) where {T<:Integer} = "both are $T integers"

f (generic function with 8 methods)

In [28]:
methods(f)

# 8 methods for generic function "f" from Main:
 [1] f(a::T, b::T) where T<:Integer
     @ In[27]:1
 [2] f(a::Integer, b::Integer)
     @ In[10]:5
 [3] f(a::Float64, b::Float64)
     @ In[8]:1
 [4] f(a::T, b::T) where T<:Number
     @ In[23]:1
 [5] f(a::Number, b::Number)
     @ In[10]:2
 [6] f(a::Number, b)
     @ In[10]:3
 [7] f(a, b::Number)
     @ In[10]:4
 [8] f(a, b)
     @ In[10]:1

In [29]:
f(big(1), big(2))

"both are BigInt integers"

In [30]:
f("foo", "bar") # <== still doesn't apply to non-numbers

"fallback"

### Varargs methods

In [31]:
f(args::Number...) = "$(length(args))-ary heterogeneous call"
f(args::T...) where {T<:Number} = "$(length(args))-ary homogeneous call"

f (generic function with 10 methods)

In [32]:
f(1)

"1-ary homogeneous call"

In [33]:
f(1, 2, 3)

"3-ary homogeneous call"

In [34]:
f(1, 1.5, 2)

"3-ary heterogeneous call"

In [35]:
f()

"0-ary homogeneous call"

In [36]:
f(1, 2) # <== previous 2-arg method is more specific

"both are Int64 integers"

In [37]:
f("foo") # <== still doesn't apply to non-numbers

LoadError: MethodError: no method matching f(::String)

[0mClosest candidates are:
[0m  f(::Any, [91m::Number[39m)
[0m[90m   @[39m [35mMain[39m [90m[4mIn[10]:4[24m[39m
[0m  f(::Any, [91m::Any[39m)
[0m[90m   @[39m [35mMain[39m [90m[4mIn[10]:1[24m[39m
[0m  f([91m::T[39m, [91m::T[39m) where T<:Integer
[0m[90m   @[39m [35mMain[39m [90m[4mIn[27]:1[24m[39m
[0m  ...


In [38]:
# "splat" (more below)
f([1, 2, 3]...)

"3-ary homogeneous call"

### Optional Arguments

In [39]:
h(x, y = 0) = 2x + 3y

h (generic function with 2 methods)

In [40]:
methods(h)

# 2 methods for generic function "h" from Main:
 [1] h(x)
     @ In[39]:1
 [2] h(x, y)
     @ In[39]:1

Shorthand for this:
```
h(x, y) = 2x + 3y
h(x) = h(x, 0)
```

### Keyword Arguments

In [41]:
k(x, y = 0; opt::Bool = false) = opt ? 2x+y : x+2y

k (generic function with 2 methods)

In [42]:
k(2)

2

In [43]:
k(2, 3)

8

In [44]:
k(2, opt=true)

4

In [45]:
k(2, 3, opt=true)

7

In [46]:
foo(x, y; req::Bool) = req ? 2x+y : x+2y

foo (generic function with 1 method)

In [47]:
foo(2, 3)

LoadError: UndefKeywordError: keyword argument `req` not assigned

In [48]:
methods(k)

# 2 methods for generic function "k" from Main:
 [1] k(x)
     @ In[41]:1
 [2] k(x, y; opt)
     @ In[41]:1

In [49]:
k(2, opt=true)

4

### Keyword arguments: slurp and splat

In [50]:
function allkw(; kw...)
    @show keys(kw)
end

allkw (generic function with 1 method)

In [51]:
allkw(a=1,b=2)

keys(kw) = (:a, :b)


(:a, :b)

Just like iterators can be splatted as positional arguments, dict-like collections and named tuples can be splatted as keyword arguments.

In [52]:
function rect(;width=1,height=1,fill="#")
    for i in 1:height
        println(fill^width)
    end
end

rect (generic function with 1 method)

In [53]:
params = (width=8,height=3,fill='A')

(width = 8, height = 3, fill = 'A')

In [54]:
rect(; params...)

AAAAAAAA
AAAAAAAA
AAAAAAAA


### Exercises

#### Exercise 1

Write a function that repeats a string an integer number of times which takes the arguments in either order.

#### Exercise 2a

Write a function `F` that returns the tuple `(x, y, k)` where:
- `x` is the first positional argument and is mandatory
- `y` is the second positional argument and is optional
- `k` is an optional keyword argument

The optional arguments should have the following defaults:
- `y` defaults to `2x`
- `k` defaults to `2y`

#### Exercise 2b

Write a function `G` just like `F` but with differet defaults:
- `k` defaults to `2x`
- `y` defaults to `2k`

## Exercise 1

Note: If using multiple dispatch, don't define with an anonymous function.

In [1]:
repeat(a::Int64, b::String) = b ^ a
repeat(a::String, b::Int64) = a ^ b

@assert repeat("ab", 3) === "ababab"
@assert repeat(3, "ab") === "ababab"

# Exercise 2a

In [4]:
F(x, y=2x; k=2y) = (x, y, k)

@assert F(1) == (1, 2, 4)
@assert F(1, 3) == (1, 3, 6)
@assert F(1, 4, k=5) == (1, 4, 5)
try
    F(1, 4, 5)
    @assert false
catch
end

# Exercise 2b

In [9]:
# Doesn't work
G(x, y=2k; k=2x) = (x, y, k)

@assert G(1) == (1, 4, 2)


UndefVarError: UndefVarError: `k` not defined